In [1]:
import os
import pandas as pd
import time
import torch
from ultralytics import YOLO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim

In [ ]:

from ultralytics.utils.benchmarks import benchmark

# Load a model

benchmark(model=r"./YOLOv8_training/experiment19/weights/best.pt",data=r"C:\Users\windows11\Downloads\CBR LNP.v1i.yolov8\data.yaml", imgsz=640, half=False, device=0)
# 모델 로드


# 테스트 실행
#results = model.val(data=r"C:\Users\windows11\Downloads\CBR LNP.v1i.yolov8\data.yaml", imgsz=640)



image 1/1 C:\Users\windows11\anaconda3\envs\yolov8\Lib\site-packages\ultralytics\assets\bus.jpg: 640x480 (no detections), 35.0ms
Speed: 14.2ms preprocess, 35.0ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 480)


val: Scanning C:\Users\windows11\Downloads\CBR LNP.v1i.yolov8\valid\labels.cache... 155 images, 0 backgrounds, 0 corrupt: 100%|██████████| 155/155 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:01<00:00, 87.18it/s]

                   all        155        174      0.862      0.859      0.927      0.864
Speed: 0.6ms preprocess, 7.6ms inference, 0.0ms loss, 1.0ms postprocess per image

PyTorch: starting from 'YOLOv8_training\experiment19\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 24, 8400) (6.0 MB)

TorchScript: starting export with torch 2.5.1+cu121...


In [7]:
from ultralytics import YOLO
import numpy as np
# YOLOv8 모델 불러오기 (사전 훈련된 모델 또는 사용자 훈련 모델 경로)
model = YOLO(r"./YOLOv8_training/experiment19/weights/best.pt")  # 또는 사용자 모델 경로: "runs/detect/train/weights/best.pt"

# 데이터셋 경로 설정 (검증용 데이터셋)
dataset_path = r"C:\Users\windows11\Downloads\CBR LNP.v1i.yolov8\data.yaml"

# 검증 실행
metrics = model.val(
    data=dataset_path,
    imgsz=640,  # 이미지 크기 축소
    batch=1,    # 배치 크기 감소
    device="cpu",  # CPU로 전환 (GPU 부족 시)
    save_json=True
)

# 결과를 딕셔너리로 변환
results = metrics.results_dict
# 결과 출력
print("Validation Results:")
print(f"mAP50: {results['metrics/mAP50']:.4f}")
print(f"mAP50-95: {results['metrics/mAP50-95']:.4f}")
print(f"Precision: {results['metrics/precision']:.4f}")
print(f"Recall: {results['metrics/recall']:.4f}")

# 결과 세부 정보 출력 (옵션)
print("\nClass-wise Metrics:")
for cls, (p, r, ap50, ap50_95) in enumerate(
    zip(metrics['metrics/per_class_precision'],
        metrics['metrics/per_class_recall'],
        metrics['metrics/per_class_AP50'],
        metrics['metrics/per_class_AP50-95'])):
    print(f"Class {cls}: Precision={p:.4f}, Recall={r:.4f}, AP50={ap50:.4f}, AP50-95={ap50_95:.4f}")


Ultralytics 8.3.40  Python-3.11.10 torch-2.5.1+cu121 CPU (12th Gen Intel Core(TM) i5-12450H)
Model summary (fused): 168 layers, 3,009,548 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\windows11\Downloads\CBR LNP.v1i.yolov8\valid\labels.cache... 155 images, 0 backgrounds, 0 corrupt: 100%|██████████| 155/155 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 155/155 [00:53<00:00,  2.92it/s]


                   all        155        174      0.862      0.859      0.927      0.865
   Chemical_respirator         34         36      0.828      0.804      0.845      0.757
                   PCR          1          1      0.431          1      0.995      0.995
             autoclave         30         30       0.93      0.882      0.945      0.881
      biohazard_symbol         25         27      0.927      0.815      0.917      0.883
          cell_culture         24         27       0.92      0.857      0.944       0.89
            centrifuge         27         28          1      0.897      0.968      0.827
       chemical_hazard         14         25          1      0.757      0.876      0.818
Speed: 2.4ms preprocess, 320.1ms inference, 0.0ms loss, 1.5ms postprocess per image
Saving runs\detect\val16\predictions.json...
Results saved to runs\detect\val16
Validation Results:


KeyError: 'metrics/mAP50'

In [2]:
def extract_label_from_filename(filename):
    """파일명에서 실험실 유형 라벨을 추출"""
    if "chem" in filename.lower():
        return "chem"
    elif "bio" in filename.lower():
        return "bio"
    elif "rad" in filename.lower():
        return "rad"
    else:
        return "unknown"
image_dir = r"./dataset_places365/data"

In [3]:
# ResNet 모델 로드
from torchvision.models import ResNet50_Weights
resnet_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# YOLO 관련 함수
def detect_objects(image_path, model):
    """YOLO 모델로 객체 탐지"""
    results = model(image_path, conf=0.5)
    detected_objects = []
    if results and hasattr(results[0], "boxes") and results[0].boxes:
        for box in results[0].boxes:
            class_id = int(box.cls)
            detected_objects.append(model.names[class_id])
    else:
        detected_objects.append("no_objects_detected")
    return detected_objects

def setup_yolo_pipeline(image_dir):
    """YOLO 파이프라인 설정"""
    yolo_model = YOLO(r"./YOLOv8_training/experiment19/weights/best.pt")
    
    # 데이터 준비
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
    data = [{"image_name": file, "label": extract_label_from_filename(file)} for file in image_files]
    labels_df = pd.DataFrame(data)

    # 객체 탐지 및 결과 저장
    detected_objects_list = []
    for image_name in labels_df['image_name']:
        image_path = os.path.join(image_dir, image_name)
        detected_objects = detect_objects(image_path, yolo_model)
        detected_objects_list.append(", ".join(detected_objects))
    
    labels_df['detected_objects'] = detected_objects_list
    labels_df = labels_df[~labels_df['detected_objects'].str.contains("no_objects_detected", na=False)]

    return yolo_model, labels_df

def train_random_forest_classifier(labels_df):
    """TF-IDF와 Random Forest로 장면 분류 모델 학습"""
    vectorizer = TfidfVectorizer(max_features=20, min_df=0.01, max_df=0.85, stop_words=['no_objects_detected'])
    tfidf_features = vectorizer.fit_transform(labels_df['detected_objects']).toarray()
    X_train, X_test, y_train, y_test = train_test_split(tfidf_features, labels_df['label'], test_size=0.2, random_state=42)

    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    print("Random Forest 분류 성능 보고서:")
    print(classification_report(y_test, y_pred))

    return vectorizer, classifier

In [11]:

# ResNet 관련 함수
def preprocess_image(image_path):
    """ResNet 입력 전처리"""
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image).unsqueeze(0)

def prepare_resnet_data(image_dir):
    """ResNet 데이터 준비"""
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
    labels = [extract_label_from_filename(file) for file in image_files]

    # 데이터 분리: 학습 데이터와 테스트 데이터
    train_files, test_files, train_labels, test_labels = train_test_split(
        image_files, labels, test_size=0.2, random_state=42
    )
    return train_files, test_files, train_labels, test_labels

def train_resnet(train_files, train_labels, image_dir):
    """ResNet 학습"""
    # 레이블 매핑
    label_to_index = {label: idx for idx, label in enumerate(set(train_labels))}
    index_to_label = {idx: label for label, idx in label_to_index.items()}

    # 모델 구성
    num_classes = len(label_to_index)
    resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)

    # 학습 루프
    resnet_model.train()
    epochs = 20
    for epoch in range(epochs):
        epoch_loss = 0
        correct = 0
        total = 0

        for file, label in zip(train_files, train_labels):
            image_path = os.path.join(image_dir, file)
            image_tensor = preprocess_image(image_path)
            label_tensor = torch.tensor([label_to_index[label]], dtype=torch.long)

            optimizer.zero_grad()
            outputs = resnet_model(image_tensor)
            loss = criterion(outputs, label_tensor)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == label_tensor).sum().item()
            total += label_tensor.size(0)

        accuracy = correct / total
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

        # 가중치 저장
        torch.save(resnet_model.state_dict(), f"resnet_model_weights_epoch{epoch + 1}.pth")
        print(f"Epoch {epoch + 1}: 모델 가중치가 'resnet_model_weights_epoch{epoch + 1}.pth'에 저장되었습니다.")

    return index_to_label


In [18]:

def evaluate_resnet(test_files, test_labels, image_dir, index_to_label):
    """ResNet 성능 평가"""
    y_true = []
    y_pred = []

    resnet_model.eval()
    with torch.no_grad():
        for file, true_label in zip(test_files, test_labels):
            image_path = os.path.join(image_dir, file)
            image_tensor = preprocess_image(image_path)

            outputs = resnet_model(image_tensor)
            _, predicted = torch.max(outputs, 1)

            # 실제 라벨과 예측된 라벨 추가
            y_true.append(true_label)
            y_pred.append(index_to_label[predicted.item()])

    # 성능 지표 계산
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_true, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    report = classification_report(y_true, y_pred, zero_division=0)

    print("ResNet 성능 보고서:")
    print(report)
    print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")


In [32]:
if __name__ == "__main__":
    # 데이터 준비
    image_dir = r"./dataset_places365/data"
    train_files, test_files, train_labels, test_labels = prepare_resnet_data(image_dir)
    

In [33]:
def check_dataset_split(train_files, test_files):
    """학습 및 테스트 데이터 크기 확인"""
    print(f"학습 데이터 개수: {len(train_files)}")
    print(f"테스트 데이터 개수: {len(test_files)}")

check_dataset_split(train_files, test_files)

학습 데이터 개수: 960
테스트 데이터 개수: 241


In [15]:

if __name__ == "__main__":
    # 데이터 준비
    image_dir = r"./dataset_places365/data"
    train_files, test_files, train_labels, test_labels = prepare_resnet_data(image_dir)

    # ResNet 학습
    index_to_label = train_resnet(train_files, train_labels, image_dir)

    # ResNet 평가
    evaluate_resnet(test_files, test_labels, image_dir, index_to_label)

Epoch 1/20, Loss: 1117.5109, Accuracy: 0.3604
Epoch 1: 모델 가중치가 'resnet_model_weights_epoch1.pth'에 저장되었습니다.
Epoch 2/20, Loss: 1076.8413, Accuracy: 0.3656
Epoch 2: 모델 가중치가 'resnet_model_weights_epoch2.pth'에 저장되었습니다.
Epoch 3/20, Loss: 1056.2228, Accuracy: 0.3729
Epoch 3: 모델 가중치가 'resnet_model_weights_epoch3.pth'에 저장되었습니다.
Epoch 4/20, Loss: 1050.5592, Accuracy: 0.3563
Epoch 4: 모델 가중치가 'resnet_model_weights_epoch4.pth'에 저장되었습니다.
Epoch 5/20, Loss: 1044.4021, Accuracy: 0.3688
Epoch 5: 모델 가중치가 'resnet_model_weights_epoch5.pth'에 저장되었습니다.
Epoch 6/20, Loss: 1024.9310, Accuracy: 0.3979
Epoch 6: 모델 가중치가 'resnet_model_weights_epoch6.pth'에 저장되었습니다.
Epoch 7/20, Loss: 1037.2267, Accuracy: 0.3854
Epoch 7: 모델 가중치가 'resnet_model_weights_epoch7.pth'에 저장되었습니다.
Epoch 8/20, Loss: 1030.8964, Accuracy: 0.3990
Epoch 8: 모델 가중치가 'resnet_model_weights_epoch8.pth'에 저장되었습니다.
Epoch 9/20, Loss: 1038.4736, Accuracy: 0.3823
Epoch 9: 모델 가중치가 'resnet_model_weights_epoch9.pth'에 저장되었습니다.
Epoch 10/20, Loss: 1051.0397, Accurac

In [19]:
def evaluate_resnet2(test_files, test_labels, image_dir, index_to_label, weights_path):
    """
    ResNet 성능 평가

    Args:
        test_files (list): 테스트 이미지 파일 목록
        test_labels (list): 테스트 라벨 목록
        image_dir (str): 이미지 파일 디렉토리
        index_to_label (dict): 인덱스를 레이블로 변환하는 딕셔너리
        weights_path (str): 저장된 가중치 파일 경로
    """
    # 저장된 가중치 로드
    resnet_model.load_state_dict(torch.load(weights_path))
    resnet_model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        for file, true_label in zip(test_files, test_labels):
            image_path = os.path.join(image_dir, file)
            image_tensor = preprocess_image(image_path)

            outputs = resnet_model(image_tensor)
            _, predicted = torch.max(outputs, 1)

            # 실제 라벨과 예측된 라벨 추가
            y_true.append(true_label)
            y_pred.append(index_to_label[predicted.item()])

    # 성능 지표 계산
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_true, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    report = classification_report(y_true, y_pred, zero_division=0)

    print("ResNet 성능 보고서:")
    print(report)
    print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")


In [20]:
if __name__ == "__main__":
    # ResNet 학습 및 평가
    weights_path=r"./resnet_model_weights_epoch8.pth"
    evaluate_resnet2(test_files, test_labels, image_dir, index_to_label, weights_path)

ResNet 성능 보고서:
              precision    recall  f1-score   support

         bio       0.00      0.00      0.00        76
        chem       0.35      0.92      0.51        87
         rad       0.21      0.04      0.07        78

    accuracy                           0.34       241
   macro avg       0.19      0.32      0.19       241
weighted avg       0.20      0.34      0.21       241

Accuracy: 0.34, Precision: 0.20, Recall: 0.34, F1 Score: 0.21


In [21]:
import time

def evaluate_resnet_with_latency(test_files, test_labels, image_dir, index_to_label, weights_path):
    """
    ResNet 성능 평가 및 레이턴시 측정

    Args:
        test_files (list): 테스트 이미지 파일 목록
        test_labels (list): 테스트 라벨 목록
        image_dir (str): 이미지 파일 디렉토리
        index_to_label (dict): 인덱스를 레이블로 변환하는 딕셔너리
        weights_path (str): 저장된 가중치 파일 경로
    """
    # 저장된 가중치 로드
    resnet_model.load_state_dict(torch.load(weights_path))
    resnet_model.eval()

    y_true = []
    y_pred = []
    latencies = []

    with torch.no_grad():
        for file, true_label in zip(test_files, test_labels):
            image_path = os.path.join(image_dir, file)
            image_tensor = preprocess_image(image_path)

            # 레이턴시 측정 시작
            start_time = time.time()
            outputs = resnet_model(image_tensor)
            end_time = time.time()
            # 레이턴시 저장
            latencies.append(end_time - start_time)

            _, predicted = torch.max(outputs, 1)

            # 실제 라벨과 예측된 라벨 추가
            y_true.append(true_label)
            y_pred.append(index_to_label[predicted.item()])

    # 성능 지표 계산
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="weighted", zero_division=0)
    recall = recall_score(y_true, y_pred, average="weighted", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    report = classification_report(y_true, y_pred, zero_division=0)

    # 레이턴시 통계
    avg_latency = sum(latencies) / len(latencies)
    print("ResNet 성능 보고서:")
    print(report)
    print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")
    print(f"평균 레이턴시: {avg_latency * 1000:.2f} ms (입력당 추론 시간)")


In [22]:
evaluate_resnet_with_latency(test_files, test_labels, image_dir, index_to_label, weights_path)


ResNet 성능 보고서:
              precision    recall  f1-score   support

         bio       0.00      0.00      0.00        76
        chem       0.35      0.92      0.51        87
         rad       0.21      0.04      0.07        78

    accuracy                           0.34       241
   macro avg       0.19      0.32      0.19       241
weighted avg       0.20      0.34      0.21       241

Accuracy: 0.34, Precision: 0.20, Recall: 0.34, F1 Score: 0.21
평균 레이턴시: 73.45 ms (입력당 추론 시간)


In [26]:
import time

def single_image_pipeline_with_latency(image_path, yolo_model, vectorizer, classifier):
    """
    단일 이미지에 대한 객체 탐지와 분류 모델 실행 시간 측정

    Args:
        image_path (str): 이미지 파일 경로
        yolo_model: YOLO 모델 객체
        vectorizer: 학습된 TF-IDF 벡터라이저
        classifier: 학습된 Random Forest 분류 모델

    Returns:
        dict: 결과와 레이턴시 정보 포함
    """
    # YOLO 객체 탐지 시간 측정
    start_time = time.time()
    detected_objects, detection_latency = detect_objects_with_latency(image_path, yolo_model)
    end_time = time.time()

    # YOLO 탐지 레이턴시
    yolo_latency = end_time - start_time

    # Random Forest 입력 데이터 생성 (TF-IDF 변환)
    start_time = time.time()
    tfidf_features = vectorizer.transform([" ".join(detected_objects)]).toarray()

    # Random Forest 분류 예측 시간 측정
    prediction = classifier.predict(tfidf_features)
    end_time = time.time()

    classification_latency = end_time - start_time

    # 총 레이턴시
    total_latency = yolo_latency + classification_latency

    return {
        "detected_objects": detected_objects,
        "prediction": prediction[0],
        "yolo_latency": yolo_latency,
        "classification_latency": classification_latency,
        "total_latency": total_latency,
    }


In [30]:
result = single_image_pipeline_with_latency(image_dir, yolo_model, vectorizer, classifier)



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1201 c:\Users\windows11\Desktop\Project \dataset_places365\data\aug_radio (1).jpg: 352x640 1 lab_animal, 36.8ms
image 2/1201 c:\Users\windows11\Desktop\Project \dataset_places365\data\aug_radio (10).jpg: 352x640 2 manipulation_arms, 31.2ms
image 3/1201 c:\Users\windows11\Desktop\Project \dataset_places365\data\aug_radio (100).jpg: 384x640 (no detections), 34.7ms
image 4/1201 c:\Users\windows11\Desktop\Project \dataset_places365\data\aug_radio (10

In [31]:
print(f"탐지된 객체: {result['detected_objects']}")
print(f"분류 결과: {result['prediction']}")
print(f"YOLO 레이턴시: {result['yolo_latency']:.4f} 초")
print(f"분류 레이턴시: {result['classification_latency']:.4f} 초")
print(f"총 레이턴시: {result['total_latency']:.4f} 초")

탐지된 객체: ['lab_animal']
분류 결과: rad
YOLO 레이턴시: 15.1627 초
분류 레이턴시: 0.0145 초
총 레이턴시: 15.1771 초
